In [1]:
import pynvml

In [1]:
#! pip install --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" apex-master/.

/root/anaconda3/envs/detection/lib/python3.7/site-packages/pip/_internal/commands/install.py:243: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/
Processing ./apex-master
Skipping wheel build for apex, due to binaries being disabled for it.
  Attempting uninstall: apex
    Found existing installation: apex 0.1
    Uninstalling apex-0.1:
      Successfully uninstalled apex-0.1
    Running setup.py install for apex ... done


In [1]:
import sys
sys.path.insert(0, "timm_efficientdet-pytorch")
sys.path.insert(0, "omegaconf")
import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob
from apex import amp
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [2]:
marking = pd.read_csv('../global-wheat-detection/train_clean.csv')

bboxs = np.stack(marking['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    marking[column] = bboxs[:,i]
marking.drop(columns=['bbox'], inplace=True)
marking.head

<bound method NDFrame.head of         Unnamed: 0   image_id  width  height     source      x      y      w  \
0                0  b6ab77fd7   1024    1024    usask_1  834.0  222.0   56.0   
1                1  b6ab77fd7   1024    1024    usask_1  226.0  548.0  130.0   
2                2  b6ab77fd7   1024    1024    usask_1  377.0  504.0   74.0   
3                3  b6ab77fd7   1024    1024    usask_1  834.0   95.0  109.0   
4                4  b6ab77fd7   1024    1024    usask_1   26.0  144.0  124.0   
...            ...        ...    ...     ...        ...    ...    ...    ...   
147572      147788  5e0747034   1024    1024  arvalis_2   64.0  619.0   84.0   
147573      147789  5e0747034   1024    1024  arvalis_2  292.0  549.0  107.0   
147574      147790  5e0747034   1024    1024  arvalis_2  134.0  228.0  141.0   
147575      147791  5e0747034   1024    1024  arvalis_2  430.0   13.0  184.0   
147576      147792  5e0747034   1024    1024  arvalis_2  875.0  740.0   94.0   

         

In [3]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

df_folds = marking[['image_id']].copy()
df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()
df_folds.loc[:, 'source'] = marking[['image_id', 'source']].groupby('image_id').min()['source']
df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['source'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
)
df_folds.loc[:, 'fold'] = 0
print(df_folds.head)
for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number
print(df_folds.head)

<bound method NDFrame.head of            bbox_count     source stratify_group  fold
image_id                                             
00333207f          55  arvalis_1    arvalis_1_3     0
005b0d8bb          20    usask_1      usask_1_1     0
006a994f7          25    inrae_1      inrae_1_1     0
00764ad5d          41    inrae_1      inrae_1_2     0
00b5fefed          25  arvalis_3    arvalis_3_1     0
...               ...        ...            ...   ...
ffb445410          57     rres_1       rres_1_3     0
ffbf75e5b          52  arvalis_1    arvalis_1_3     0
ffbfe7cc0          34  arvalis_1    arvalis_1_2     0
ffc870198          41    usask_1      usask_1_2     0
ffdf83e42          39  arvalis_1    arvalis_1_2     0

[3373 rows x 4 columns]>
<bound method NDFrame.head of            bbox_count     source stratify_group  fold
image_id                                             
00333207f          55  arvalis_1    arvalis_1_3     2
005b0d8bb          20    usask_1      usask_1_1   

/root/anaconda3/envs/detection/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [4]:
def get_train_transforms():
    return A.Compose(
        [
            A.RandomSizedCrop(min_max_height=(800, 800), height=1024, width=1024, p=0.5), # Crop a random part of the input and rescale it to (1024, 1024).
            A.OneOf([#Select one of transforms to apply with p=0.9
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                     val_shift_limit=0.2, p=0.9),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.9),
            ],p=0.9),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=1024, width=1024, p=1),#Resize the input to the given height and width.
            A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc', #[x_min, y_min, x_max, y_max]
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=1024, width=1024, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

In [5]:
TRAIN_ROOT_PATH = '../global-wheat-detection/train'

class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()

        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        
        if self.test or random.random() > 0.5:
            image, boxes = self.load_image_and_boxes(index)
        else:
            image, boxes = self.load_cutmix_image_and_boxes(index)

        # there is only one class
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    break

        return image, target, image_id
    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        records = self.marking[self.marking['image_id'] == image_id]
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        return image, boxes
    
    def load_cutmix_image_and_boxes(self, index, imsize=1024):
        """ 
        This implementation of cutmix author:  https://www.kaggle.com/nvnnghia 
        Refactoring and adaptation: https://www.kaggle.com/shonenkov
        """
        w, h = imsize, imsize
        s = imsize // 2
    
        xc, yc = [int(random.uniform(imsize * 0.25, imsize * 0.75)) for _ in range(2)]  # center x, y
        indexes = [index] + [random.randint(0, self.image_ids.shape[0] - 1) for _ in range(3)]

        result_image = np.full((imsize, imsize, 3), 1, dtype=np.float32)
        result_boxes = []

        for i, index in enumerate(indexes):
            image, boxes = self.load_image_and_boxes(index)
            if i == 0:
                x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
            elif i == 1:  # top right
                x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
                x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
            elif i == 2:  # bottom left
                x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, max(xc, w), min(y2a - y1a, h)
            elif i == 3:  # bottom right
                x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)
            result_image[y1a:y2a, x1a:x2a] = image[y1b:y2b, x1b:x2b]
            padw = x1a - x1b
            padh = y1a - y1b

            boxes[:, 0] += padw
            boxes[:, 1] += padh
            boxes[:, 2] += padw
            boxes[:, 3] += padh

            result_boxes.append(boxes)

        result_boxes = np.concatenate(result_boxes, 0)
        np.clip(result_boxes[:, 0:], 0, 2 * s, out=result_boxes[:, 0:])
        result_boxes = result_boxes.astype(np.int32)
        result_boxes = result_boxes[np.where((result_boxes[:,2]-result_boxes[:,0])*(result_boxes[:,3]-result_boxes[:,1]) > 0)]
        return result_image, result_boxes

In [6]:
fold_number = 0

train_dataset = DatasetRetriever(
    image_ids=df_folds[df_folds['fold'] != fold_number].index.values,
    marking=marking,
    transforms=get_train_transforms(),
    test=False,
)

validation_dataset = DatasetRetriever(
    image_ids=df_folds[df_folds['fold'] == fold_number].index.values,
    marking=marking,
    transforms=get_valid_transforms(),
    test=True,
)

In [7]:
image, target, image_id = train_dataset[1]
boxes = target['boxes'].cpu().numpy().astype(np.int32)  # torch.float64 -> int32

numpy_image = image.permute(1,2,0).cpu().numpy() # torch.Size([3, 512, 512]), torch.float32 -> (512, 512, 3), float32

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(numpy_image, (box[1], box[0]), (box[3],  box[2]), (0, 1, 0), 2)
    
ax.set_axis_off()
ax.imshow(numpy_image)

In [8]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [9]:
import warnings

warnings.filterwarnings("ignore")

class Fitter:
    
    def __init__(self, model, device, config):
        self.config = config
        self.epoch = 0

        self.base_dir = f'./{config.folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5

        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.model = model.to(device) # added
        
        self.model, self.optimizer = amp.initialize(self.model, self.optimizer, opt_level="O1", verbosity=0)
        #self.amp_state = amp.state_dict()
            
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.log(f'Fitter prepared. Device is {self.device}')
    
    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss = self.train_one_epoch(train_loader)

            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/last-checkpoint.bin')

            t = time.time()
            summary_loss = self.validation(validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-checkpoint-*epoch.bin'))[:-3]:
                    os.remove(path)

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1

    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        t = time.time()
        for step, (images, targets, image_ids) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                images = torch.stack(images)
                batch_size = images.shape[0]
                images = images.to(self.device).float()
                target_res = {} # added
                boxes = [target['boxes'].to(self.device).float() for target in targets]
                labels = [target['labels'].to(self.device).float() for target in targets]
                
                target_res['bbox'] = boxes # added
                target_res['cls'] = labels # added
                target_res["scale"] = torch.tensor([1.0] * batch_size, dtype=torch.float).to(self.device) #added
                target_res["img_size"] = torch.tensor([images[0].shape[-2:]] * batch_size, dtype=torch.float).to(self.device)#added
                
                #loss, _, _ = self.model(images, boxes, labels) command out
                output = self.model(images, target_res) #added
                loss = output['loss'] #added
                summary_loss.update(loss.detach().item(), batch_size)
                
        return summary_loss
    
    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        t = time.time()
        for step, (images, targets, image_ids) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    
                    print(
                        
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            
            
            images = torch.stack(images)
            images = images.to(self.device).float()
            
            batch_size = images.shape[0]
            #add target_res
            target_res = {}
            boxes = [target['boxes'].to(self.device).float() for target in targets]
            labels = [target['labels'].to(self.device).float() for target in targets]

            target_res['bbox'] = boxes
            target_res['cls'] = labels 

            #self.optimizer.zero_grad() ori
            
            
            #self.scheduler.step(metrics = summary_loss.avg)#added
            #loss, _, _ = self.model(images, boxes, labels)
            output = self.model(images, target_res)
            
            #loss = output['loss'] ori
            loss = output['loss']
            
            #loss.backward()

            #summary_loss.update(loss.detach().item(), batch_size)

            #self.optimizer.step()

            #if self.config.step_scheduler:
            #    self.scheduler.step()
           
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
            

            if (step+1) % self.config.accumulate == 0:
                #summary_loss.update(loss.detach().item(), batch_size * self.config.accumulate)
                self.optimizer.step()
                self.optimizer.zero_grad()
            summary_loss.update(loss.detach().item(), batch_size)
            #summary_loss.update(loss.detach().item(), batch_size) 
            
            #if self.config.step_scheduler:
            #    self.scheduler.step()
            #summary_loss.update(loss.detach().item(), batch_size) 
        return summary_loss

    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
            'amp': amp.state_dict()
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        amp.load_state_dict(checkpoint['amp'])
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')



In [10]:
class TrainGlobalConfig:
    num_workers = 4
    batch_size = 2 
    n_epochs = 80 # n_epochs = 40
    lr = 0.0002
    mixed_precision = True
    accumulate = 16
    # how many batches to make the gradient accumulation e.g batch_size = 2 & accumulate = 16 -> real batch is 2*16 =32
    folder = 'effdet5-cutmix-augmix-1024-3'

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = True  # do scheduler.step after validation stage loss

#     SchedulerClass = torch.optim.lr_scheduler.OneCycleLR
#     scheduler_params = dict(
#         max_lr=0.001,
#         epochs=n_epochs,
#         steps_per_epoch=int(len(train_dataset) / batch_size),
#         pct_start=0.1,
#         anneal_strategy='cos', 
#         final_div_factor=10**5
#     )
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=2,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )
    # --------------------

In [11]:
def collate_fn(batch):
    return tuple(zip(*batch))

def run_training():
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    #device = torch.device('cuda:0')
    net.to(device)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        sampler=RandomSampler(train_dataset),
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
        collate_fn=collate_fn,
    )
    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=TrainGlobalConfig.batch_size,
        num_workers=TrainGlobalConfig.num_workers,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
        collate_fn=collate_fn,
    )
    
    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
    fitter.load('effdet5-cutmix-augmix-1024-3/last-checkpoint.bin')
    fitter.fit(train_loader, val_loader)

In [12]:
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet

def get_net():
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)
    checkpoint = torch.load('weights/tf_efficientdet_d5-ef44aea8.pth')
    #checkpoint = torch.load('effdet5-cutmix-augmix-1024/last-checkpoint.bin')
    net.load_state_dict(checkpoint)
    config.num_classes = 1
    config.image_size = 1024
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    return DetBenchTrain(net, config)
net = get_net()

In [ ]:
#new

In [ ]:
run_training()

Fitter prepared. Device is cuda

2020-08-01T10:15:19.083490
LR: 0.0002
[RESULT]: Train. Epoch: 0, summary_loss: 24.82099, time: 1376.49510
[RESULT]: Val. Epoch: 0, summary_loss: 11.03965, time: 96.21419

2020-08-01T10:39:53.106540
LR: 0.0002
[RESULT]: Train. Epoch: 1, summary_loss: 7.38931, time: 1364.80135
[RESULT]: Val. Epoch: 1, summary_loss: 3.22785, time: 93.92708

2020-08-01T11:04:13.214892
LR: 0.0002
[RESULT]: Train. Epoch: 2, summary_loss: 2.38089, time: 1362.21152
[RESULT]: Val. Epoch: 2, summary_loss: 1.45749, time: 94.04129

2020-08-01T11:28:30.756489
LR: 0.0002
[RESULT]: Train. Epoch: 3, summary_loss: 1.26275, time: 1365.82638
[RESULT]: Val. Epoch: 3, summary_loss: 0.92477, time: 93.88570

2020-08-01T11:52:51.883151
LR: 0.0002
[RESULT]: Train. Epoch: 4, summary_loss: 0.86599, time: 1366.05050
[RESULT]: Val. Epoch: 4, summary_loss: 0.70382, time: 94.00107

2020-08-01T12:17:16.412618
LR: 0.0002
[RESULT]: Train. Epoch: 5, summary_loss: 0.69669, time: 1363.05236
[RESULT]: Val. 

In [ ]:
run_training()

Fitter prepared. Device is cuda

2020-08-01T14:00:37.992572
LR: 0.0002
[RESULT]: Train. Epoch: 7, summary_loss: 0.55544, time: 1370.69087
[RESULT]: Val. Epoch: 7, summary_loss: 0.49537, time: 95.92041

2020-08-01T14:25:05.939469
LR: 0.0002
[RESULT]: Train. Epoch: 8, summary_loss: 0.52259, time: 1366.69037
[RESULT]: Val. Epoch: 8, summary_loss: 0.48068, time: 93.64748

2020-08-01T14:49:27.614850
LR: 0.0002
[RESULT]: Train. Epoch: 9, summary_loss: 0.49243, time: 1356.06678
[RESULT]: Val. Epoch: 9, summary_loss: 0.46648, time: 93.67366

2020-08-01T15:13:38.693238
LR: 0.0002
[RESULT]: Train. Epoch: 10, summary_loss: 0.47793, time: 1367.19257
[RESULT]: Val. Epoch: 10, summary_loss: 0.45958, time: 93.46925

2020-08-01T15:38:00.695620
LR: 0.0002
[RESULT]: Train. Epoch: 11, summary_loss: 0.46059, time: 1375.56610
[RESULT]: Val. Epoch: 11, summary_loss: 0.45432, time: 93.89035

2020-08-01T16:02:31.502545
LR: 0.0002
[RESULT]: Train. Epoch: 12, summary_loss: 0.44753, time: 1364.27015
[RESULT]: Va

In [ ]:
run_training()

Fitter prepared. Device is cuda

2020-08-02T10:54:11.083554
LR: 1.25e-05
[RESULT]: Train. Epoch: 40, summary_loss: 0.36582, time: 1375.38452


In [ ]:
run_training()

Fitter prepared. Device is cuda

2020-08-02T12:04:11.822188
LR: 1.25e-05
[RESULT]: Train. Epoch: 41, summary_loss: 0.35429, time: 1366.14057
[RESULT]: Val. Epoch: 41, summary_loss: 0.41361, time: 95.55043

2020-08-02T12:28:34.186543
LR: 1.25e-05
[RESULT]: Train. Epoch: 42, summary_loss: 0.36431, time: 1361.50584
[RESULT]: Val. Epoch: 42, summary_loss: 0.40801, time: 93.37925

2020-08-02T12:52:52.900490
LR: 6.25e-06
[RESULT]: Train. Epoch: 43, summary_loss: 0.35841, time: 1364.80911
[RESULT]: Val. Epoch: 43, summary_loss: 0.40815, time: 93.38552

2020-08-02T13:17:11.801814
LR: 6.25e-06


In [ ]:
run_training()

Fitter prepared. Device is cuda

2020-08-02T13:48:02.097189
LR: 6.25e-06
[RESULT]: Train. Epoch: 44, summary_loss: 0.34294, time: 1376.48549
[RESULT]: Val. Epoch: 44, summary_loss: 0.41814, time: 95.93674

2020-08-02T14:12:35.223768
LR: 6.25e-06
[RESULT]: Train. Epoch: 45, summary_loss: 0.35160, time: 1374.95744
[RESULT]: Val. Epoch: 45, summary_loss: 0.41474, time: 93.72856

2020-08-02T14:37:04.591578
LR: 6.25e-06


In [ ]:
run_training()

Fitter prepared. Device is cuda

2020-08-02T14:39:39.457791
LR: 6.25e-06
[RESULT]: Train. Epoch: 46, summary_loss: 0.33650, time: 1372.31932
[RESULT]: Val. Epoch: 46, summary_loss: 0.42312, time: 95.96924

2020-08-02T15:04:08.431670
LR: 6.25e-06
[RESULT]: Train. Epoch: 47, summary_loss: 0.34475, time: 1368.00799
[RESULT]: Val. Epoch: 47, summary_loss: 0.41939, time: 93.62275

2020-08-02T15:28:33.556546
LR: 3.125e-06


In [ ]:
run_training()

Fitter prepared. Device is cuda

2020-08-02T15:41:16.012579
LR: 6.25e-06
[RESULT]: Train. Epoch: 48, summary_loss: 0.32997, time: 1372.50762
[RESULT]: Val. Epoch: 48, summary_loss: 0.42837, time: 95.63350

2020-08-02T16:05:44.850364
LR: 3.125e-06
[RESULT]: Train. Epoch: 49, summary_loss: 0.33741, time: 1366.01010
[RESULT]: Val. Epoch: 49, summary_loss: 0.42474, time: 93.21401

2020-08-02T16:30:04.773110
LR: 3.125e-06


In [ ]:
run_training()

Fitter prepared. Device is cuda

2020-08-02T16:33:04.800063
LR: 3.125e-06
[RESULT]: Train. Epoch: 50, summary_loss: 0.32334, time: 1367.43870
[RESULT]: Val. Epoch: 50, summary_loss: 0.43107, time: 95.57833

2020-08-02T16:57:28.524825
LR: 3.125e-06
[RESULT]: Train. Epoch: 51, summary_loss: 0.33352, time: 1364.51792
[RESULT]: Val. Epoch: 51, summary_loss: 0.42778, time: 93.08148

2020-08-02T17:21:49.912094
LR: 3.125e-06
[RESULT]: Train. Epoch: 52, summary_loss: 0.35275, time: 1353.48812
[RESULT]: Val. Epoch: 52, summary_loss: 0.41706, time: 93.19944

2020-08-02T17:45:57.312197
LR: 1.5625e-06
[RESULT]: Train. Epoch: 53, summary_loss: 0.35783, time: 1365.92849
[RESULT]: Val. Epoch: 53, summary_loss: 0.41417, time: 93.08658

2020-08-02T18:10:17.030593
LR: 1.5625e-06
[RESULT]: Train. Epoch: 54, summary_loss: 0.35487, time: 1366.02145
[RESULT]: Val. Epoch: 54, summary_loss: 0.41361, time: 93.08964

2020-08-02T18:34:36.853498
LR: 1.5625e-06
[RESULT]: Train. Epoch: 55, summary_loss: 0.35150, ti

In [ ]:
#old

In [ ]:
run_training()

Fitter prepared. Device is cuda

2020-07-26T06:00:40.404498
LR: 0.0002
[RESULT]: Train. Epoch: 0, summary_loss: 24.84400, time: 1352.99052
[RESULT]: Val. Epoch: 0, summary_loss: 10.56032, time: 95.48311

2020-07-26T06:24:50.257250
LR: 0.0002
[RESULT]: Train. Epoch: 1, summary_loss: 7.34619, time: 1355.67438
[RESULT]: Val. Epoch: 1, summary_loss: 3.08379, time: 93.08489

2020-07-26T06:49:03.414578
LR: 0.0002
[RESULT]: Train. Epoch: 2, summary_loss: 2.38021, time: 1356.62710
[RESULT]: Val. Epoch: 2, summary_loss: 1.39102, time: 93.10240

2020-07-26T07:13:17.538840
LR: 0.0002
[RESULT]: Train. Epoch: 3, summary_loss: 1.23982, time: 1357.94241
[RESULT]: Val. Epoch: 3, summary_loss: 0.91983, time: 92.95035

2020-07-26T07:37:29.853014
LR: 0.0002
[RESULT]: Train. Epoch: 4, summary_loss: 0.85040, time: 1361.37497
[RESULT]: Val. Epoch: 4, summary_loss: 0.67968, time: 93.20256

2020-07-26T08:01:45.868613
LR: 0.0002
[RESULT]: Train. Epoch: 5, summary_loss: 0.69379, time: 1367.40230
[RESULT]: Val. 

In [15]:
run_training()

Fitter prepared. Device is cuda

2020-07-27T13:06:40.288137
LR: 2.5e-05
[RESULT]: Train. Epoch: 34, summary_loss: 0.37702, time: 1366.15344
[RESULT]: Val. Epoch: 34, summary_loss: 0.40462, time: 95.61121

2020-07-27T13:31:02.828048
LR: 2.5e-05
[RESULT]: Train. Epoch: 35, summary_loss: 0.37663, time: 1364.76111
[RESULT]: Val. Epoch: 35, summary_loss: 0.40451, time: 93.45869

2020-07-27T13:55:24.564084
LR: 1.25e-05
[RESULT]: Train. Epoch: 36, summary_loss: 0.37581, time: 1356.87973
[RESULT]: Val. Epoch: 36, summary_loss: 0.40382, time: 93.45562

2020-07-27T14:19:35.613501
LR: 1.25e-05
[RESULT]: Train. Epoch: 37, summary_loss: 0.37582, time: 1365.29286
[RESULT]: Val. Epoch: 37, summary_loss: 0.40428, time: 93.29906

2020-07-27T14:43:55.002632
LR: 6.25e-06
[RESULT]: Train. Epoch: 38, summary_loss: 0.37379, time: 1363.46426
[RESULT]: Val. Epoch: 38, summary_loss: 0.40407, time: 93.28827

2020-07-27T15:08:12.443200
LR: 6.25e-06
[RESULT]: Train. Epoch: 39, summary_loss: 0.37356, time: 1365.62

KeyboardInterrupt: 